In [145]:
import pandas as pd
from glob import glob

## New Zealand
- https://electionresults.govt.nz/electionresults_2023/
- https://electionresults.govt.nz/electionresults_2023/statistics/votes-by-voting-place-electorate-index.html

In [146]:
year = 2023
election = "general"

In [147]:
output = []
for filepath in glob("../data/New Zealand/2023/general/*.csv"):
    frag = pd.read_csv(filepath, header=2)
    frag = frag.rename(columns={"Unnamed: 0": "town", "Unnamed: 1": "polling_address"})
    electorate = frag[frag["polling_address"].fillna("#").str.contains(" Total")]["polling_address"].values[0].split(" Total")[0]

    frag_party = frag.copy()
    frag_party = frag_party.loc[frag_party[frag_party["polling_address"] == "Party"].index[0] + 1:]
    frag_party = frag_party.rename(columns={"town": "candidate", "polling_address": "party"})
    frag_party = frag_party[["candidate", "party"]]
    frag_party = pd.concat([
            frag_party, 
            pd.DataFrame([{"candidate": "Informal Candidate Votes", "party": "Informal Candidate Votes"}])
        ], 
        ignore_index=True
    )

    n = frag[frag["town"] == "Voting Places"].index
    n = n[0] if len(n) > 0 else 999
    frag = frag.head(n)

    frag = frag.dropna(subset=[list(frag)[5]])
    frag["town"] = frag["town"].ffill()
    frag = frag.melt(id_vars=["town", "polling_address"], var_name="candidate")
    frag = pd.merge(frag, frag_party, on="candidate")
    frag["value"] = frag["value"].astype(int)
    frag = frag[~frag["polling_address"].str.contains("Total")]
    frag["electorate"] = electorate
    frag["polling_id"] = frag["polling_address"].str.lower().str.replace(" ", "-").str.replace(",", "") + "-" + frag["town"] + "-" + electorate

    output.append(frag)

df = pd.concat(output, ignore_index=True)
df.head()

,town,polling_address,candidate,value,party,electorate,polling_id
0,Albany,"Westfield Mall, Centre Court, 219 Don McKinnon...","COKER, Christopher Hamish",0,Aotearoa Legalise Cannabis Party,Auckland Central,westfield-mall-centre-court-219-don-mckinnon-d...
1,Auckland City,"Atrium on Elliot, Shop 925, next to Rebel Spor...","COKER, Christopher Hamish",19,Aotearoa Legalise Cannabis Party,Auckland Central,atrium-on-elliot-shop-925-next-to-rebel-sport-...
2,Auckland City,"Auckland City Mission, 140 Hobson Street","COKER, Christopher Hamish",1,Aotearoa Legalise Cannabis Party,Auckland Central,auckland-city-mission-140-hobson-street-Auckla...
3,Auckland City,"Auckland University of Technology, Level 4, Bu...","COKER, Christopher Hamish",2,Aotearoa Legalise Cannabis Party,Auckland Central,auckland-university-of-technology-level-4-buil...
4,Auckland City,"Auckland University Students Association, 4 Al...","COKER, Christopher Hamish",1,Aotearoa Legalise Cannabis Party,Auckland Central,auckland-university-students-association-4-alf...


In [156]:
tt.sort_values(["polling_id"]).head(50)

,polling_id,candidate,party,rate,value,rank
0,209-hilton-meeting-room-by-robert-harris-cafe-...,"BALLANTYNE, Robert Andrew",New Zealand First Party,0.032810,23,3
7,209-hilton-meeting-room-by-robert-harris-cafe-...,"THOMAS, Karl Raymond",New Conservatives,0.011412,8,6
6,209-hilton-meeting-room-by-robert-harris-cafe-...,"SHEARER, Wayne",New Zealand Loyal,0.012839,9,5
5,209-hilton-meeting-room-by-robert-harris-cafe-...,"MEAGER, James Rawiri",National Party,0.680456,477,1
8,209-hilton-meeting-room-by-robert-harris-cafe-...,"Van AMERSFOORT, Adolf Pieter",New Nation Party,0.000000,0,8
3,209-hilton-meeting-room-by-robert-harris-cafe-...,Informal Candidate Votes,Informal Candidate Votes,0.001427,1,7
2,209-hilton-meeting-room-by-robert-harris-cafe-...,"GILCHRIST, Barbara",Green Party,0.028531,20,4
1,209-hilton-meeting-room-by-robert-harris-cafe-...,"CLARKSON, Michael David",Rock the Vote NZ,0.000000,0,8
4,209-hilton-meeting-room-by-robert-harris-cafe-...,"LUXTON, Jo-Anne Marie",Labour Party,0.232525,163,2
9,209-hilton-meeting-room-by-robert-harris-cafe-...,"FERRIS, Tākuta",Te Pāti Māori,0.437500,7,1


# New Zealand
Data extracted from https://electionresults.govt.nz/

In [149]:
tt = df.groupby(["polling_id", "candidate", "party"]).agg({"value": "sum"})
tt["rate"] = tt.groupby(level=[0], group_keys=False).apply(lambda x: x/x.sum()).fillna(0)
tt = tt.reset_index()
tt = tt[["polling_id", "candidate", "party", "rate", "value"]]

tt["rank"] = tt.groupby(["polling_id"])["value"].rank("min", ascending=False).astype(int)

In [150]:
tt.to_csv(f"../data_output/New Zealand/{year}_{election}.csv.gz", compression="gzip", index=False)

In [151]:
df_electorates = pd.read_html("https://en.wikipedia.org/wiki/New_Zealand_electorates")[1]
df_electorates.columns = [x.lower() for x in df_electorates.columns]
df_electorates = df_electorates[["electorate", "region"]]
df_electorates.head()

,electorate,region
0,Auckland Central,Auckland
1,Banks Peninsula,Canterbury
2,Bay of Plenty,Bay of Plenty
3,Botany,Auckland
4,Christchurch Central,Canterbury


In [152]:
set(df["electorate"]) - set(df_electorates["electorate"])

{'Hauraki-Waikato',
 'Ikaroa-Rāwhiti',
 'Te Tai Hauāuru',
 'Te Tai Tokerau',
 'Te Tai Tonga',
 'Tāmaki Makaurau',
 'Waiariki'}

In [153]:
df_location = pd.merge(df, df_electorates, on="electorate", how="outer")
df_location = df_location[["polling_id", "town", "polling_address", "electorate", "region"]].drop_duplicates().reset_index(drop=True)
df_location.to_csv(f"../data_output/New Zealand/{year}_{election}_location.csv.gz", compression="gzip", index=False)

In [154]:
df_location

,polling_id,town,polling_address,electorate,region
0,westfield-mall-centre-court-219-don-mckinnon-d...,Albany,"Westfield Mall, Centre Court, 219 Don McKinnon...",Auckland Central,Auckland
1,atrium-on-elliot-shop-925-next-to-rebel-sport-...,Auckland City,"Atrium on Elliot, Shop 925, next to Rebel Spor...",Auckland Central,Auckland
2,auckland-city-mission-140-hobson-street-Auckla...,Auckland City,"Auckland City Mission, 140 Hobson Street",Auckland Central,Auckland
3,auckland-university-of-technology-level-4-buil...,Auckland City,"Auckland University of Technology, Level 4, Bu...",Auckland Central,Auckland
4,auckland-university-students-association-4-alf...,Auckland City,"Auckland University Students Association, 4 Al...",Auckland Central,Auckland
...,...,...,...,...,...
2788,tolaga-bay-area-school-24-resolution-street-To...,Tolaga Bay,"Tolaga Bay Area School, 24 Resolution Street",East Coast,Gisborne and Bay of Plenty
2789,caledonian-hall-60-king-street-Whakatāne-East ...,Whakatāne,"Caledonian Hall, 60 King Street",East Coast,Gisborne and Bay of Plenty
2790,te-koputu-library-&-exhibition-centre-49-kakah...,Whakatāne,"Te Koputu Library & Exhibition Centre, 49 Kaka...",East Coast,Gisborne and Bay of Plenty
2791,te-whare-wānanga-o-awanuiārangi-124-mcalister-...,Whakatāne,"Te Whare Wānanga o Awanuiārangi, 124 McAlister...",East Coast,Gisborne and Bay of Plenty


In [155]:
pd.merge(tt, df_location
         , on="polling_id").groupby(["region", "polling_id"]).count()

candidate  \
region                polling_id                                                      
Auckland              ahutoetoe-school-ruxton-road-Milldale-Kaipara k...          7   
                      al-madinah-school-8-westney-road-Māngere-Māngere            8   
                      albany-community-hub-575-albany-highway-Albany-...          6   
                      albany-community-hub-575-albany-highway-Albany-...          7   
                      all-saints-anglican-church-284-ponsonby-road-Po...         11   
...                                                                             ...   
West Coast and Tasman richmond-mall-next-to-boulevard-café-queen-stre...         10   
                      saxton-pavilion-nelson-cricket-ground-142-saxto...         10   
                      st-john-station-8-smith-street-Reefton-West Coa...         10   
                      tai-poutini-polytechnic-conference-room-73-87-t...         10   
                      tākaka-anglican-church-hall-42-commercial-stree...         10   

                                                                          party  \
region                polling_id                                                  
Auckland              ahutoetoe-school-ruxton-road-Milldale-Kaipara k...      7   
                      al-madinah-school-8-westney-road-Māngere-Māngere        8   
                      albany-community-hub-575-albany-highway-Albany-...      6   
                      albany-community-hub-575-albany-highway-Albany-...      7   
                      all-saints-anglican-church-284-ponsonby-road-Po...     11   
...                                                                         ...   
West Coast and Tasman richmond-mall-next-to-boulevard-café-queen-stre...     10   
                      saxton-pavilion-nelson-cricket-ground-142-saxto...     10   
                      st-john-station-8-smith-street-Reefton-West Coa...     10   
                      tai-poutini-polytechnic-conference-room-73-87-t...     10   
                      tākaka-anglican-church-hall-42-commercial-stree...     10   

                                                                          rate  \
region                polling_id                                                 
Auckland              ahutoetoe-school-ruxton-road-Milldale-Kaipara k...     7   
                      al-madinah-school-8-westney-road-Māngere-Māngere       8   
                      albany-community-hub-575-albany-highway-Albany-...     6   
                      albany-community-hub-575-albany-highway-Albany-...     7   
                      all-saints-anglican-church-284-ponsonby-road-Po...    11   
...                                                                        ...   
West Coast and Tasman richmond-mall-next-to-boulevard-café-queen-stre...    10   
                      saxton-pavilion-nelson-cricket-ground-142-saxto...    10   
                      st-john-station-8-smith-street-Reefton-West Coa...    10   
                      tai-poutini-polytechnic-conference-room-73-87-t...    10   
                      tākaka-anglican-church-hall-42-commercial-stree...    10   

                                                                          value  \
region                polling_id                                                  
Auckland              ahutoetoe-school-ruxton-road-Milldale-Kaipara k...      7   
                      al-madinah-school-8-westney-road-Māngere-Māngere        8   
                      albany-community-hub-575-albany-highway-Albany-...      6   
                      albany-community-hub-575-albany-highway-Albany-...      7   
                      all-saints-anglican-church-284-ponsonby-road-Po...     11   
...                                                                         ...   
West Coast and Tasman richmond-mall-next-to-boulevard-café-queen-stre...     10   
                      saxton-pavilion-nelson-cri